In [1]:
import os
import sys
import pickle
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchtext.vocab import GloVe
from underthesea import word_tokenize
from torch.utils.data import DataLoader

In [2]:
PROJECT_ROOT = os.path.expanduser('~/vietnamese-poem-generation')
sys.path.append(PROJECT_ROOT)
sys.path.insert(0, PROJECT_ROOT)
from constants import *
sys.path.append(UTILS_DIR)
sys.path.append(MODEL_DIR)
from tokenization import *
from dataset import *
from transformer import *
from train import *

2025-02-26 13:03:41.679031: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740549821.693559   14920 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740549821.697894   14920 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-26 13:03:41.713052: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Poem Data

In [3]:
df = pd.read_csv(os.path.join(DATA_DIR, 'poem_dataset.csv'))
df.head()

,content,title,url,genre
0,ngày đông se sắt lạnh trong lòng\ncó việc đi n...,SAY NẮNG,https://www.facebook.com/groups/48640773509859...,7 chu
1,ôm đàn thao thức đến nữa đêm\nréo rắt cung âm ...,NaN,https://www.facebook.com/groups/17645444269765...,7 chu
2,tết có người vui có kẻ buồn\nngười cười toe to...,TẾT HAI THÁI CỰC,https://www.facebook.com/groups/17645444269765...,7 chu
3,đã quá ba mươi mộng lỡ làng\nđi tìm day dứt mả...,TRÁI NGANG ĐỨC HẠNH,https://www.facebook.com/groups/48640773509859...,7 chu
4,mai đào nở rộ đón nàng xuân\nsợi nắng hanh vàn...,DÁNG XUÂN,https://www.facebook.com/groups/17645444269765...,7 chu


In [4]:
print(df['content'][0])

ngày đông se sắt lạnh trong lòng
có việc đi ngang qua chỗ đó
bắt gặp ánh mắt ai say đắm
nụ cười ai khiến tôi hoảng loạn
ngày hôm nay tôi lướt qua ai
thấy tim mình xốn xang loạn nhịp
sao thế có phải mình say nắng
chiều đông đâu có nắng mà say
say ngất say ngây say không tỉnh
lâng lâng như uống phải men tình
thả hồn theo gió lộng mênh mông
mộng mơ bồng bềnh khi sương xuống
mây chiều đã nhuộm đỏ hoàng hôn
chim khôn đã cùng nhau về tổ
ngất ngây đắm chìm say gì thế
mà đứng chôn chân chả muốn về
sương nhẹ rơi vương đầy trên cỏ
sao lòng buồn bỗng thấy bâng khuâng
giật mình chợt tỉnh cơn mộng mị
tỉnh lại đi tôi ơi tỉnh lại
còn các con đang ngóng chờ cửa
còn bạn đời đang đợi tôi về


# Create Dataset for Poem Data

- Build vocabulary from the text data:

In [5]:
vocab = build_vocabulary(df)

In [6]:
with open(os.path.join(STORAGE_DIR, 'poem_vocab.pkl'), 'wb') as f:
    pickle.dump(vocab, f)
print('Vocabulary saved as pkl file')

Vocabulary saved as pkl file


In [4]:
with open(os.path.join(STORAGE_DIR, 'poem_vocab.pkl'), 'rb') as f:
    vocab = pickle.load(f)
print('Vocabulary loaded from pkl file')

Vocabulary loaded from pkl file


- Create poem dataset: In this notebook, I just get about **1000 poems** for training example.

In [5]:
train_dataset = PoemDataset(df=df[:1000], tokenizer=word_tokenize, vocab=vocab)

In [7]:
with open(os.path.join(STORAGE_DIR, 'poem_dataset.pkl'), 'wb') as f:
    pickle.dump(train_dataset, f)
print('Dataset saved as pkl file')

Dataset saved as pkl file


In [5]:
# load dataset
with open(os.path.join(STORAGE_DIR, 'poem_dataset.pkl'), 'rb') as f:
    train_dataset = pickle.load(f)
print('Dataset loaded from pkl file')

Dataset loaded from pkl file


- Create data loader:

In [6]:
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

# Training


In [7]:
VOCAB_SIZE = len(vocab)
EMBEDDING_DIMS = 100
HIDDEN_DIMS = 100
N_LAYERS = 2
N_HEADS = 4
DROPOUT = 0.2
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
LEARNING_RATE = 0.001
N_EPOCHS = 100

For pre-trained embeddings, I used GloVe with 100-dimensional embeddings:

In [8]:
glove = GloVe(name="6B", dim=100)
pretrained_embedding = glove.vectors

In [9]:
model = TransformerModel(
    vocab_size=VOCAB_SIZE,
    emb_size=EMBEDDING_DIMS,
    num_encoder_layers=N_LAYERS,
    nhead=N_HEADS,
    dim_feedforward=HIDDEN_DIMS,
    dropout=DROPOUT,
    device=DEVICE,
    pretrained_embedding=pretrained_embedding,
    freeze_embedding=True
)

CRITERION = nn.CrossEntropyLoss()
OPTIMIZER = optim.AdamW(model.parameters(), lr=LEARNING_RATE)
SCHEDULER = torch.optim.lr_scheduler.StepLR(OPTIMIZER, 1, gamma=0.95)

In [10]:
train_model(
    model=model,
    train_loader=train_loader,
    criterion=CRITERION,
    optimizer=OPTIMIZER,
    scheduler=SCHEDULER,
    num_epochs=N_EPOCHS,
    device=DEVICE,
    model_name='poem_transformer'
)

Epoch 1/100:   5%|▌         | 1167/23307 [00:39<12:29, 29.56it/s, loss=3.0351]


KeyboardInterrupt: 